This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [10]:
# First, import the relevant modules
import requests
import json
from pandas.io.json import json_normalize

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
apiK = 'GfzgHivF6WaPZv91sv-5'
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
key= dict(api_key=apiK, start_date= '2017-01-01', end_date = '2017-01-02')
req= requests.get(url, params= key)  

In [46]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json= req.json()
#json
#column = (json['dataset_data']['column_names'][0:])
data =(json['dataset_data']['data'][0][1:])
#dict(zip(column,data))

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# My Solution Starts here

# Call the API and initialize all needed Variables to collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

# Convert returned JSON object into a Python dictionary

In [197]:
#Assign the API Key
apiK = 'GfzgHivF6WaPZv91sv-5'
#Assign the URL
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
#Set the parameters for API call.
key= dict(api_key=apiK, start_date= '2017-01-01', end_date = '2017-12-31')
#Submit a get request to the API
req= requests.get(url, params= key) 
#Convert the returned JSON object into a Python dictionary
json= req.json()
#Define Dict to store daily open price with key as Date
daily_open= {}
#Define Dict to store daily high with key as Date
daily_high= {}
#Define Dict to store daily low with key as Date
daily_low={}
#Define List to store daily trading volume
trading_vol=[]
#Define Dict to store daily difference between high and low with key as Date
dly_delta={}
#Get list of columns for daily data
col_list = (json['dataset_data']['column_names'][0:])


# Iterate through the converted Json dictionary to get all values

In [198]:
#iterate through list of data for whole year to get all values
for i in range (len(json['dataset_data']['data'])):
    #data =(json['dataset_data']['data'][i][0:])
    #print(data) #tocheck the position values for each value
#Set the key to date of that day
    key=json['dataset_data']['data'][i][0]
    #Yearly_data[key]=dict(zip(column,data))
    #print (Yearly_data[key]) to check column value mapping
#Get the open price for that date and store in Dict
    daily_open[key]= json['dataset_data']['data'][i][1]
#Get the high for that date and store in Dict
    daily_high[key]= json['dataset_data']['data'][i][2]
#Get the low for that date and store in Dict
    daily_low[key]= json['dataset_data']['data'][i][3]
#Get difference between high and low for that date and store in Dict
    dly_delta[key]= daily_high[key]-daily_low[key]
#Append the Trading Volume for that date to trading volume list
    trading_vol.append(json['dataset_data']['data'][i][7])
#Append the diff between closing value for 2 consecutive dates and store in list
    if i == 0:
        two_day_delta=[]
    else:
        two_day_delta.append(abs(json['dataset_data']['data'][i-1][4]- json['dataset_data']['data'][i][4]))

# Create Median function

In [200]:
def median(lst):
    n = len(lst)
    s = sorted(lst)
    return (sum(s[n//2-1:n//2+1])/2.0, s[n//2])[n % 2] if n else None

# Print all asked data

In [199]:
#remove NONE values by filtering out only for float
daily_open = {key: value for (key,value) in daily_open.items() if type(value) == float}
#Sort the open price dictionary
Open_price_Sorted = sorted(daily_open.items(), key=lambda x: x[1],  reverse=True)
dly_delta = sorted(dly_delta.items(),key=lambda x:x[1], reverse= True)
print('Highest Opening price was on Date:Value {0}').format(Open_price_Sorted[0])
print('Lowest Opening price was on Date:Value {0}').format(Open_price_Sorted[-1])
print ('The largest change in any one day was on Date:Value {0}'.format(dly_delta[0]))
print ('Largest change between any two days was: {0}').format(max(two_day_delta))
print ('Average daily trading volume during 2017 year was: {0}').format(sum(trading_vol)/len(trading_vol))
print ('Median daily trading volume during 2017 year was: {0}').format(median(trading_vol))

Highest Opening price was on Date:Value (u'2017-12-14', 53.11)
Lowest Opening price was on Date:Value (u'2017-01-24', 34.0)
The largest change in any one day was on Date:Value (u'2017-05-11', 2.8100000000000023)
Largest change between any two days was: 2.56
Average daily trading volume during 2017 year was: 3853589.45098
Median daily trading volume during 2017 year was: 3292223.0


# # My Solution Ends here